In [1]:
## import libraries
import sys
import glob
import re

import geopandas as gpd
import cartopy
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import cmocean.cm as cmo

# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature

# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from plotter import draw_basemap

pd.options.display.float_format = "{:,.2f}".format # makes it so pandas tables display only first two decimals

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/eaton-scripps/share/proj failed


In [2]:
path_to_data = '/data/projects/Comet/cwp140/' 
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [18]:
HUC8_ID = 14080107
## load PRISM watershed precip dataset
fname = path_to_data + 'preprocessed/PRISM/PRISM_HUC8_CO.nc'
PRISM = xr.open_dataset(fname)
PRISM = PRISM.sel(HUC8=str(HUC8_ID))
PRISM = PRISM.where(PRISM.extreme == 1, drop=True)

## load final trajectory dataset
fname = '/home/dnash/comet_data/preprocessed/ERA5_trajectories/final/PRISM_HUC8_{0}.nc'.format(HUC8_ID)
traj = xr.open_dataset(fname)

## load watershed shapefile
## use geopandas to import the shapefile
fp = path_to_data + 'downloads/CO_HUC8/wbdhu8.shp'
polys = gpd.read_file(fp, crs="epsg:4326") # have to manually set the projection

In [22]:
## total extreme precip
total_prec = PRISM.prec.sum('date')

## get all the dates from the trajectory dataset where AR Scale > 0
ar_days = traj.where(traj.ar_scale > 0, drop=True).start_date.values
## select those dates from precip dataset
tmp = PRISM.sel(date=ar_days)

## calculate fraction of extreme precipitation related to ARs for each watershed
ar_prec = tmp.prec.sum('date')

frac = (ar_prec/total_prec)*100
frac

<xarray.DataArray 'prec' ()>
array(9.33226575)
Coordinates:
    HUC8     <U8 '14080107'

In [ ]:
## plot choropleth map with watersheds showing the % of extreme precip related to ARs